<a href="https://colab.research.google.com/github/sdr0598/ml-drug-discovery/blob/main/week1_pytorch_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 - Day 1: PyTorch Fundamentals
## Key Concepts
- Tensor: multi-dimensional array that can live on GPU
- requires_grad: tells PyTorch to track operations for differentiation
- loss.backward(): computes gradients automatically
- w.grad.zero_(): must zero gradients each step or they accumulate

## Drug Discovery Connection
- Rows of a 2D tensor = molecules
- Columns = features (descriptors, fingerprints)
- The training loop is how all models (SVM was manual, NN is automatic) learn

In [1]:
import torch
print(torch.__version__)
print("GPU available:", torch.cuda.is_available())

2.9.0+cpu
GPU available: False


In [2]:
# Tensors are arrays but smarter
# Same concept as NumPy arrays but can run on GPU and track gradients
import torch

# Create tensors
x = torch.tensor([1.0, 2.0, 3.0])
print(x)
print(x.shape)
print(x.dtype)

# 2D tensor (think: a matrix of molecular descriptors)
# Rows = molecules, Columns = features
descriptors = torch.zeros(10, 100)
print(descriptors.shape)

tensor([1., 2., 3.])
torch.Size([3])
torch.float32
torch.Size([10, 100])


In [12]:
# Autograd: Makes PyTorch special
# requires_grad=True tells PyTorch: "track everything done to this tensor"
# This is how neurals networks learn - automatic differentiation

w = torch.tensor(2.0, requires_grad=True) # model weight??
x = torch.tensor(3.0)

y = w * x + 1 # y = 2 * 3 + 1 = 7
print("y =", y)

y.backward()
print("dy/dw", w.grad) #



y = tensor(7., grad_fn=<AddBackward0>)
dy/dw tensor(3.)


In [15]:
# Prototype of "Binding Affinity Predictor"
# Input: One molecule descriptor value
# Output: predicted affinity
# Goal: Find the best weight w that minimizes prediction error

torch.manual_seed(42)

# Dummy "true" relationship: affinity = 0.5 * descriptor + noise
true_w = 0.5
X = torch.randn(20) # Tensor: 1 row, 20 columns; rows = features; columns = molecules
y_true = true_w*X + torch.randn(20)*0.1

# Start model with a random weight
w = torch.tensor(0.0, requires_grad=True)

#Training loop = core of ALL neural network training
learning_rate = 0.1
for epoch in range(20):
  # Forward pass: make prediction
  y_pred = w *X

  # Compute loss (how far off are we?)
  loss = ((y_pred - y_true) **2).mean()

  # Backward pass: compute gradients
  loss.backward()

  # Update weight (gradient descent)
  with torch.no_grad():
  # Changes made to w does not count as a forward pass
  # no_grad() ensures this
  # Otherwise, this would be recorded into the computation graph and PyTorch would try to differentiate it
    w -= learning_rate * w.grad

  # Zero gradients (Important - must do this every step)
  w.grad.zero_()

  if epoch % 5 == 0:
    print(f"Epoch {epoch}: loss={loss.item():.4f}, w={w.item():.4f}")

print(f"\nLearned w={w.item():.4f}, true w={true_w}")


tensor(0.2649, grad_fn=<MeanBackward0>)
tensor(0.1001, requires_grad=True)
Epoch 0: loss=0.2649, w=0.1001
tensor(0.0366, grad_fn=<MeanBackward0>)
tensor(0.3744, requires_grad=True)
Epoch 5: loss=0.0366, w=0.3744
tensor(0.0102, grad_fn=<MeanBackward0>)
tensor(0.4677, requires_grad=True)
Epoch 10: loss=0.0102, w=0.4677
tensor(0.0072, grad_fn=<MeanBackward0>)
tensor(0.4993, requires_grad=True)
Epoch 15: loss=0.0072, w=0.4993

Learned w=0.5088, true w=0.5
